In [ ]:
#### Dependencies ####
import numpy as np
import logging
import os
import sys
import time
import joblib
import matplotlib.pyplot as plt
import torch
import json
import cortex # This dependency is pycortex, which enables the plotting of flatmaps. It can be disabled.
from cvxopt import matrix, solvers # Only necessary for the stacked model.
from transformers import AutoTokenizer, AutoModelForCausalLM # Only necessary for feature extraction.
import subprocess
from tqdm.autonotebook import tqdm, trange

# Repository imports
from ridge_utils.ridge import bootstrap_ridge
import ridge_utils.npp
from ridge_utils.util import make_delayed
from ridge_utils.dsutils import make_word_ds
from ridge_utils.DataSequence import DataSequence
from ridge_utils.tokenization_helpers import generate_efficient_feat_dicts_opt
from ridge_utils.tokenization_helpers import convert_to_feature_mats_opt

# Topic model imports
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # For generating embeddings
from sklearn.decomposition import PCA # To speed up the UMAP
from sklearn.feature_extraction.text import CountVectorizer 
from bertopic.vectorizers import ClassTfidfTransformer
import en_core_web_sm
from bertopic.representation import PartOfSpeech, KeyBERTInspired, MaximalMarginalRelevance